In [1]:
import re
import os
import sys
import json
import openai
import ast

import warnings
import numpy as np
import pandas as pd
from openai import OpenAI
import logging

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)
sys.path.append('/home/or/dev/latin_music_statistics/crawler/data_dir')
sys.path.append(os.path.abspath(os.path.join(parent_dir, 'statistics_dir')))

from config import OPENAI_MODEL, THEME_PROMPT, SENTIMENT_PROMPT, BATCH_THEME_JSOL_FILE, BATCH_SENTIMENT_JSOL_FILE, DATA_JSOL_DIR, FINAL_DATA_FILE
df = pd.read_csv('/home/or/dev/latin_music_statistics/crawler/data_dir/final_data.csv', encoding='utf-8')

In [2]:
OPENAI_KEY = os.getenv('OPENAI_KEY')                                                                                                                                          
client = OpenAI(api_key = OPENAI_KEY)
SENTIMENT_PROMPT='analyis the overall sentiment in terms of:\nneural, positive, negative\nfor the following song:\n\n{song}\n\nwrite only in json format with the appropriate decimal percentage. Make sure the sentiment percentage sum to 1.'


In [38]:
from config import OPENAI_MODEL, SENTIMENT_PROMPT, FINAL_DATA_FILE

def prepare_chatgpt_msg(curr_text, prev_text, chatgpt_messages):
    chatgpt_messages.append({"role": "user", "content": prev_text})
    chatgpt_messages.append({"role": "user", "content": curr_text})
    return chatgpt_messages

def chatgpt_request_extraction(msgs, model_name, client):
    response = client.chat.completions.create(model=model_name, messages=msgs)
    return response

def extract_valid_dict(response_text, model_name, client, logger, max_attempts=5):
    attempt = 0
    chatgpt_messages = []

    while attempt < max_attempts:
        try:
            extracted_dict = ast.literal_eval(response_text)
            if isinstance(extracted_dict, dict):
                return extracted_dict
                
        except (SyntaxError, ValueError):
            logger.info(f"Attempt: {attempt + 1} to extract the Python dictionary from the text")
            attempt += 1
            prev_content = f"The previous {attempt + 1} attempt to extract of the valid Python dictionary from the text failed.\n" \
                            f"The previous result was: {response_text}\n" \
                            "Extract valid Python dictionary with no additional text, formatting, or code. The output should look exactly like this: {\"...\"}.\n" \
                            "Provide only the dictionary, nothing else.\nMake sure that one of the sentiments has the highest value\n" \
                            "Make sure the sentiment percentage sum to 1."
            
            chatgpt_messages = prepare_chatgpt_msg(response_text, prev_content, chatgpt_messages)
            extraction_response = chatgpt_request_extraction(chatgpt_messages, model_name, client)
            response_text = extraction_response.choices[0].message.content
    
    raise ValueError("Failed to extract a valid Python list after multiple attempts.")

def get_dominant_sentiment(sentiment_dict_input):
    try:
        sentiment_dict = eval(sentiment_dict_input)
    except TypeError as e:
        sentiment_dict = sentiment_dict_input

   # print(sentiment_dict)
    if sentiment_dict['positive'] == sentiment_dict['negative']:
        #print(sentiment_dict_input)
        print('1')
        dominant_sentiment = 'neutral'
    else:
        dominant_sentiment = max(sentiment_dict, key=sentiment_dict.get)
    return dominant_sentiment


def calc_sentiment(df, dict_sentiment_col, select_sentiment_col):
    df[select_sentiment_col] = df[dict_sentiment_col].apply(get_dominant_sentiment)


def find_sentiment(df, song_lyrics_col, tgt_col, client, logger, model_name, sentiment_prompt):
    for ii, (idx, row) in enumerate(df.iterrows()):
        prompt = sentiment_prompt.format(song=row[song_lyrics_col])
        chatgpt_msg_request = prepare_chatgpt_msg(prompt, '', [])
        response = chatgpt_request_extraction(chatgpt_msg_request, model_name, client)
        response_text = response.choices[0].message.content
        
        extracted_dict = extract_valid_dict(response_text, model_name, client, logger, max_attempts=5)
        df.loc[idx, tgt_col] = str(extracted_dict)

    calc_sentiment(df, 'sentiment', 'selected_sentiment')

def openai_sentiment_finder(df, song_lyrics_col='lyrics', tgt_col='sentiment'):
    model_name = OPENAI_MODEL
    sentiment_prompt = SENTIMENT_PROMPT
    OPENAI_KEY = os.getenv('OPENAI_KEY')                                                                                                                                          
    client = OpenAI(api_key = OPENAI_KEY)                                                                                                                                   

    # Establish logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler()          # Log messages to the console (stdout)
        ]
    )
    logger = logging.getLogger(__name__)

    find_sentiment(df, song_lyrics_col, tgt_col, client, logger, OPENAI_MODEL, SENTIMENT_PROMPT)

In [50]:
model_name = OPENAI_MODEL
sentiment_prompt = SENTIMENT_PROMPT

# Establish logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()          # Log messages to the console (stdout)
    ]
)
logger = logging.getLogger(__name__)

song_lyrics_col = 'lyrics'
tgt_col = 'sentiment2'


for ii, (idx, row) in enumerate(dffinal.iterrows()):
    if row['selected_sentiment2'] != 'neutral':
        continue
    print(idx)        
    prompt = sentiment_prompt.format(song=row[song_lyrics_col])
    chatgpt_msg_request = prepare_chatgpt_msg(prompt, '', [])
    response = chatgpt_request_extraction(chatgpt_msg_request, model_name, client)
    response_text = response.choices[0].message.content
    
    extracted_dict = extract_valid_dict(response_text, model_name, client, logger, max_attempts=5)
    print(extracted_dict)
    dffinal.loc[idx, tgt_col] = str(extracted_dict)

    # if ii ==10:
    #     break

calc_sentiment(dffinal, tgt_col, 'selected_sentiment2')


1


2024-09-21 19:42:17,075 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 19:42:17,081 - __main__ - INFO - Attempt: 1 to extract the Python dictionary from the text
2024-09-21 19:42:17,859 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'positive': 0.6, 'neutral': 0.3, 'negative': 0.1}
39


2024-09-21 19:42:18,627 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 19:42:18,638 - __main__ - INFO - Attempt: 1 to extract the Python dictionary from the text
2024-09-21 19:42:19,481 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'neutral': 0.2, 'positive': 0.3, 'negative': 0.5}
108


...

{'neutral': 0.5, 'positive': 0.3, 'negative': 0.2}
816


2024-09-21 19:43:57,502 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 19:43:57,505 - __main__ - INFO - Attempt: 1 to extract the Python dictionary from the text
2024-09-21 19:43:58,756 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'neutral': 0.3, 'positive': 0.4, 'negative': 0.3}
826


2024-09-21 19:43:59,763 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 19:43:59,769 - __main__ - INFO - Attempt: 1 to extract the Python dictionary from the text
2024-09-21 19:44:00,552 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'neutral': 0.1, 'positive': 0.6, 'negative': 0.3}
1
1
1
1
1
1
1
1
1


In [88]:
df = pd.read_csv('/home/or/dev/latin_music_statistics/crawler/data_dir/final_data.csv', encoding='utf-8')
senti = 'negative'
sdf = df[df['selected_sentiment'] ==  senti]
for ii, (idx, row) in enumerate(sdf.sample(10).iterrows()):
        print(f'------------------------------- {ii, idx} --------------------------')
        print(row['title_with_artists'])        
        print(row['theme'])
        print(row['sentiment'])

------------------------------- (0, 98) --------------------------
Donde Estan Esos Amigos by El Chaval de la Bachata
The realization of who true friends are when facing financial hardship and loss.
{'positive': 0.1, 'neutral': 0.2, 'negative': 0.7}
------------------------------- (1, 792) --------------------------
Corazón Anestesiado by Milly Quezada
The deep pain and betrayal felt after a devastating breakup, feeling as if the heart is numb while grappling with the emotional turmoil of lost love and the insincerity of the partner who hurt them.
{'neutral': 0.05, 'positive': 0.1, 'negative': 0.85}
------------------------------- (2, 451) --------------------------
Preguntando by ​sP Polanco (Ft. 24 Horas, Joell & Mickey Then)
Reflecting on the pain and confusion of a relationship that is growing distant despite mutual love.
{'neutral': 0.2, 'positive': 0.1, 'negative': 0.7}
------------------------------- (3, 209) --------------------------
Ya No Me Quieres Querer by Antony Santos
He